# Machine Learning Project - Rock Paper Scissors - Alessandro Bottoni
## Chapter 0: Data Acquisition & Settings
Data have been downloaded and stored in the data/raw folder. There are three subfolders: one for 'Rock' (726 images), one for 'Paper' (710 images) and one for 'Scissors' (752 images), for a grand total of 2188 images.


In the following cell we configure base settings, imports and set a reproducible seed.

In [12]:
# Config
RAW_DIR = "data/raw"          # contains subfolders rock/, paper/, scissors/
OUT_DIR = "data/processed"    # where we'll write train/val/test
IMG_SIZE = (128, 128)         # (H, W)
TEST_SIZE = 0.15              # final held-out test
VAL_SIZE = 0.15               # validation (from the remaining after test)
SEED = 42
OVERWRITE = True

# Imports
from pathlib import Path
import shutil, random, json
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# Reproducibility
import os
def set_seeds(seed=SEED): # we already set SEED = 42
    os.environ["PYTHONHASHSEED"] = str(seed) # controls how python randomizes hash values
    random.seed(seed) # fixes the seed for python built in random module
    np.random.seed(seed) # same but for numpy random generation

set_seeds()
Path(OUT_DIR).mkdir(parents=True, exist_ok=True) #



ModuleNotFoundError: No module named 'sklearn'